In [2]:
from osrsbox import monsters_api, items_api
from dash import Dash, html, dcc, callback, Output, Input
from dash.exceptions import PreventUpdate
import numpy as np
import plotly.express as px
import pandas as pd
import sys

In [14]:
# These don't have a rarity right now

all_items = items_api.load()


def get_all_items(database):
    """Returns a list of all unique monsters and their levels for the monster choice dropdown."""
    item_options = [
        item.name for item in all_items if item.name is not None and item.equipable
    ]
    item_options = sorted(set(item_options))
    item_options = [{"label": f"{item}", "value": item} for item in item_options]
    return item_options


truncated_items = get_all_items(all_items)

{'label': "'24-carat' sword", 'value': "'24-carat' sword"}
{'label': "'perfect' necklace", 'value': "'perfect' necklace"}
{'label': "'perfect' ring", 'value': "'perfect' ring"}
{'label': '10th squad sigil', 'value': '10th squad sigil'}
{'label': '20th anniversary boots', 'value': '20th anniversary boots'}
{'label': '20th anniversary bottom', 'value': '20th anniversary bottom'}
{'label': '20th anniversary cape', 'value': '20th anniversary cape'}
{'label': '20th anniversary gloves', 'value': '20th anniversary gloves'}
{'label': '20th anniversary hat', 'value': '20th anniversary hat'}
{'label': '20th anniversary necklace', 'value': '20th anniversary necklace'}
{'label': '20th anniversary top', 'value': '20th anniversary top'}
{'label': '3rd age amulet', 'value': '3rd age amulet'}
{'label': '3rd age axe', 'value': '3rd age axe'}
{'label': '3rd age bow', 'value': '3rd age bow'}
{'label': '3rd age cloak', 'value': '3rd age cloak'}
{'label': '3rd age druidic cloak', 'value': '3rd age druidic 

In [3]:
monsters = monsters_api.load()
initial_monster = "zulrah"
initial_item = "tanzanite fang"

NUM_KILLS = 2000
NUM_SIMULATIONS = 1000
NUM_BINS = 100
DESIRED_RATE = 0.8

if __name__ == "__main__":
    all_db_monsters = monsters_api.load()
    # Search for the monster
    srch_monstr = next(
        (
            monster
            for monster in all_db_monsters
            if monster.name.lower() == initial_monster.lower()
        ),
        None,
    )

    if srch_monstr:
        print(f"{srch_monstr.name} found in the database.")
    else:
        print(f"{initial_monster} not found in the database.")
        sys.exit(1)

    # Search for the item
    srch_item = next(
        (
            item
            for item in srch_monstr.drops
            if item.name.lower() == f"{initial_item.lower()}"
        ),
        None,
    )

    if srch_item:
        print(f"Name: {srch_item.name}, Rarity: 1/{int(1/srch_item.rarity)}")
    else:
        print(f"{initial_item} not found in the {srch_monstr}'s database.")
        sys.exit(1)

    results = []

    for _ in range(NUM_SIMULATIONS):
        trials_to_success = np.random.geometric(srch_item.rarity, NUM_KILLS)
        results.append(trials_to_success)

    results_array = np.array(results)

    results_array_reshaped = results_array.reshape(NUM_SIMULATIONS, NUM_KILLS, 1)

    cumulative_probability = np.mean(
        results_array_reshaped <= np.arange(1, NUM_KILLS + 1), axis=1
    )

    data = {
        "Number of Kills": np.arange(1, NUM_KILLS + 1),
        "Cumulative Probability": np.mean(cumulative_probability, axis=0),
    }
    df = pd.DataFrame(data)

    filt_df = df[df["Cumulative Probability"] > DESIRED_RATE]

    if not filt_df.empty:
        kill_threshold = filt_df.iloc[0, 0]
        print(
            f"You have an {DESIRED_RATE*100}% chance of receiving your item at {kill_threshold} kills."
        )
    else:
        print(
            f"No occurrence found where cumulative probability is greater than {DESIRED_RATE} for {NUM_KILLS} kills."
        )

Zulrah found in the database.
Name: Tanzanite fang, Rarity: 1/1024
You have an 80.0% chance of receiving your item at 1649 kills.


In [3]:
fig = px.line(
    df,
    x="Number of Kills",
    y="Cumulative Probability",
    title=f"CDF of getting a {srch_item.name} from {srch_monstr.name} after {NUM_KILLS} kills",
    width=700,
    height=500,
)
fig.update_layout(title_x=0.5)
fig.show()